# Raster

The pygrass interface for raster maps is divided in 4 classes that represent different ways to interact with rasters. In order to give greater freedom of implementation to users and developers, each class
uses a different C API, providing the tools to fit different needs. 

The **RasterRow** class reads the content of the raster row by row and writes it in a sequential mode: row after row. 
To read and write the same map at the same time is not supported by the RasterRow class. 

The **RasterRowIO** class implements a row cache that allows users to read and re-read raster rows randomly. 

The **RasterSegment** class divides the map into several tiles, each tile is saved into a file. With this class it is possible to read and write the pixel value randomly at the same time in the same map. 

The **RasterNumpy** class inherits from a numpy.memmap class and allows users to interact with the map as numpy matrix. 

All the Raster classes shared common methods to open, read and get raster information and write meta-data like: categories and history. Similarly to Vector class, the same syntax has been used to instantiate a raster object.


In [ ]:
from __future__ import (nested_scopes, generators, division, absolute_import,
                        with_statement, print_function, unicode_literals)

## Read raster map

In [ ]:
from grass.pygrass.raster import RasterRow

In [ ]:
elev = RasterRow('elevation')

In [ ]:
elev.exist()

In [ ]:
elev.mapset

### Open a raster map

In [ ]:
elev.open('r')

In [ ]:
elev[0][:5]

In [ ]:
elev[0]/100.

In [ ]:
import numpy as np
np.sin(elev[0])

### Query a raster using a Point object

Return Region object in Bbox format

In [ ]:
from grass.pygrass.gis.region import Region
reg = Region()
reg_bbox = reg.get_bbox()

Create two point object

In [ ]:
from grass.pygrass.vector import geometry
poi1 = geometry.Point(643521.76, 223101.31)
poi2 = geometry.Point(223101.31, 643521.76)

In [ ]:
print(elev.get_value(poi1))
print(elev.get_value(poi2))

Check if contained in the region

In [ ]:
print(reg_bbox.contains(poi1))
print(reg_bbox.contains(poi2))

## Write a raster map

### Write a new raster map from existing one

Read the *elevation* map and write new map with value divided by 10

In [ ]:
new = RasterRow('newele')
new.open('w', overwrite=True)
for row in elev:
    # the row is a Buffer class to load it
    # grass.pygrass.raster.buffer.Buffer
    new.put_row(row/10)
new.close()

Check if the new raster map exists

In [ ]:
new.exist()

### Write a new raster map starting from scratch

To create a raster from scratch you have to create row using a **Buffer** istance, so you open new raster map in write mode

In [ ]:
from grass.pygrass.raster.buffer import Buffer

# create the new raster and open it in write mode
newscratch = RasterRow('newscratch')
newscratch.open('w', overwrite=True)

now you are going to write cell by cell for each row

In [ ]:
# create an empty row
newrow = Buffer((reg.cols,), mtype='CELL')
    
# we create a raster to fill all the GRASS GIS region
for r in range(reg.rows):
    newrow[:] = np.random.random_integers(0, 1000, size=newrow.size)
    # Add row to the raster
    newscratch.put_row(newrow)

Finally you have to close the map

In [ ]:
newscratch.close()

In [ ]:
from show import show

In [ ]:
show(newscratch.name.encode())

Check if the new map exist and the range of data

In [ ]:
newscratch = RasterRow('newscratch')
newscratch.open('r')
print(newscratch.exist())
print(newscratch.info)
newscratch.close()

# Object contained in a Raster map

## Info

The **Info** class contain all the information about a raster map

In [ ]:
elev.info.mapset

In [ ]:
eleinfo = elev.info

Now you can obtain several informations

In [ ]:
print(eleinfo.range)
print(eleinfo.mapset)
print(eleinfo.south)

## History

The **History** class contain the information about history of a map

In [ ]:
elev.open()
elehisto = elev.hist

Now you can obtain several informations like creator, date of creation, keywords etc.

In [ ]:
print(elehisto.creator)
#print(elehisto.date)
print(elehisto.keyword)

# Categories

The **Category** class contain the information about categories of raster, if they are present

In [ ]:
land = RasterRow('landuse')
land.open()
land.cats

Now you can obtain more information about categories

In [ ]:
landcats = land.cats
print(landcats.labels())
print(landcats.ncats())

You could also rewrite the cats using a rules files (for example newrules.txt) containg:

**landcats.write_rules(newrules.txt)**

# Summary

We saw how:

* **read** and query a Raster
* **write** a raster: reading from existing one and from the scratch
* the **Info** element for information of a raster
* the **History** element for history of a raster
* the **Category** element for categories if they exist

# Exercise

## Exercise 1

Write a function that do the same operation of:
    
    r.mapcalc expression="new = if(elevation>100, elevation, 0)"

not using Modules class

## Exercise 2

Write a function that generate a random vector points map and query the raster map and add the value of the raster map to the attributes table of the vector map